In [1]:
from astropy.table import Table, vstack, Column, MaskedColumn, pprint
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Gaussian1DKernel

from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages

import os
import requests
import sys
sys.path.append('/Users/s2537809/DESI/desispec/py')
sys.path.append('/Users/s2537809/DESI/desimodel/py')
sys.path.append('/Users/s2537809/DESI/desitarget/py')
sys.path.append('/Users/s2537809/DESI/desiutil/py')

#from desispec.io import read_spectra
# import DESI related modules - 
from desimodel.footprint import radec2pix      # For getting healpix values
import desispec.io                             # Input/Output functions related to DESI spectra
from desispec import coaddition                # Functions related to coadding the spectra


from scipy import interpolate
import astropy.units as u
path='/Users/s2537809/Documents/DESI-STACKING'#specify code directory
os.chdir(path)
import stack_code as sc
import importlib
import fnmatch
import extinction as ext
from scipy.optimize import curve_fit
from astropy.coordinates import SkyCoord
from astropy.modeling import models
from astropy.table import Table
from matplotlib import gridspec
import spectres
import time
import warnings


import warnings
from astropy.modeling import models
from astropy import units as u
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum
from specutils.analysis import equivalent_width


#pip install lmfit, extinction, sfdmap, spectres

import pandas as pd
crossmatch = Table.read('crossmatch_classifications.fits', format = 'fits' )

/Users/s2537809/DESI/desispec/py/desispec/qproc/qextract.py:19: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def numba_extract(image_flux,image_var,x,hw=3) :
/Users/s2537809/DESI/desispec/py/desispec/image_model.py:27: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def numba_proj(image,x,sigma,flux) :
/Users/s2537809/DESI/desispec/py/desispec/preproc.py:379: Num

In [2]:
""" 
Remove the objects with failed redshift classifications.
First applying their ZWARN flag, then with the photo-z quality flag.
"""

# Apply ZWARN Flag cut
crossmatch = crossmatch[crossmatch['ZWARN'] == 0]

# Create photo-z quality flag
crossmatch['Z_CHECK']= (crossmatch['Z']-crossmatch['Z_BEST_1'])/(1+crossmatch['Z'])
crossmatch['Z_FAILURE'] = 0  
crossmatch['Z_FAILURE'][abs(crossmatch['Z_CHECK']) > 0.1] = 1 

# Apply photo-z quality flag
crossmatch = crossmatch[crossmatch['Z_FAILURE'] == 0]
print(len(crossmatch))
print(len(crossmatch[crossmatch['Z'] < 0]))
print(len(crossmatch[crossmatch['Overall_class'] == 'SFG']))

# Remove objects with no SFR or Mass
crossmatch = crossmatch[crossmatch['SFR_cons'] != -99]
crossmatch = crossmatch[crossmatch['Mass_cons'] != -99] 


print(len(crossmatch))

7533
27
5547
7461


## Cell below verifies if Spectra are downloaded and downloads if they aren't

Only run if unsure if spectra are downloaded, to save time!

In [ ]:


Overall_class = 'SFG'
my_table = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'][1000:1200]
search_column_name = 'TARGETID'
search_values = np.array(my_table[f'{search_column_name}'])



def select_data(table, column_name, search_values):
    # Get the column data as a numpy array
    column_data = table[column_name]

    SURVEY = []
    PROGRAM = []
    HEALPIX = []
    n_values = []
    Z = []
    

    for search_value in search_values:
        # Find the indices where the condition is met (value == search_value)
        indices = np.where(column_data == search_value)[0]


        if len(indices) > 0:
            # Find the corresponding Survey, program, healpix, and Z values
            SURVEY.append(table['SURVEY'][indices[0]])
            PROGRAM.append(table['PROGRAM'][indices[0]])
            HEALPIX.append(table['HEALPIX'][indices[0]])
            Z.append(table['Z'][indices[0]])
            HEALPIX_str = str(table['HEALPIX'][indices[0]])
            n_str = HEALPIX_str[:3]
            n_values.append(int(n_str))
        else:
            # Handle the case where the search value was not found
            SURVEY.append(None)
            PROGRAM.append(None)
            HEALPIX.append(None)
            n_values.append(None)
            Z.append(None)
    

    return np.array(SURVEY), np.array(PROGRAM), np.array(HEALPIX), np.array(n_values), np.array(Z), 


SURVEY = select_data(my_table, search_column_name, search_values)[0]
PROGRAM = select_data(my_table, search_column_name, search_values)[1]
HEALPIX = select_data(my_table, search_column_name, search_values)[2]
n = select_data(my_table, search_column_name, search_values)[3]
Z = select_data(my_table, search_column_name, search_values)[4]
print (SURVEY,PROGRAM,HEALPIX, n, Z)

for n, survey, program, healpix, Z, search_value in zip(n, SURVEY, PROGRAM, HEALPIX, Z, search_values):
        #download the file from the given Target ID
        target_url = f"https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/{survey}/{program}/{n}/{healpix}/coadd-{survey}-{program}-{healpix}.fits"
        
        # Define the folder name and file name
        folder_name = 'DESI_Downloads'
        file_name = f"coadd-{survey}-{program}-{healpix}.fits"
        print(search_value)


        # Check if the folder exists, and create it if it doesn't
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Check if the file exists in the folder
        file_path = os.path.join(folder_name, file_name)
        if os.path.exists(file_path):
            print(f"{file_name} already exists in {folder_name}. No need to download.")
        else:
            # File doesn't exist, download it
            try:
                response = requests.get(target_url)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        file.write(response.content)
                    print(f"{file_name} downloaded to {folder_name}.")
                else:
                    print(f"Failed to download {file_name}. Status code: {response.status_code}")
            except Exception as e:
                print(f"An error occurred while downloading {file_name}: {str(e)}")

      

# Wavelength Overlap Regions Weighted Average


In [3]:
def inv_var_weighted_average(flux1, ivar1, flux2, ivar2):
    """
    Finds the inverse varience weighted average of two flux spectra
    
    """
    weight1 = 1 / ivar1
    weight2 = 1 / ivar2
    total_weight = weight1 + weight2
    weighted_flux1 = flux1 * (weight1 / total_weight)
    weighted_flux2 = flux2 * (weight2 / total_weight)
    weighted_avg = weighted_flux1 + weighted_flux2
    return weighted_avg

def no_overlap_f(waveb,fluxb,waver,fluxr,wavez,fluxz):
    """
    Isolates the flux and wavelength arrays of each band to non-overlapping wavelength sections
    
    """
    new_b_length = np.where(waveb<waver[0])
    b_wave = waveb[new_b_length]
    b_flux = fluxb[new_b_length]
    new_r_length = np.where((waver>waveb[-1])&(waver<wavez[0]))
    r_wave = waver[new_r_length]
    r_flux = fluxr[new_r_length]
    new_z_length = np.where(wavez>waver[-1])
    z_wave = wavez[new_z_length]
    z_flux = fluxz[new_z_length]

    return b_wave,b_flux,r_wave,r_flux,z_wave,z_flux


def overlap_av(wave1,wave2,flux1,flux2,ivar1,ivar2):
    """
    Finds the overlap of two wavelength ranges.
    Computes the inverse var weighted average flux and wavelength for th eoverlapping range 
    
    """
    overlap_idx_1 = np.where(wave1 >= wave2[0]) 
    overlap_wave = wave1[overlap_idx_1]
    overlap_flux_1 = flux1[overlap_idx_1]
    overlap_ivar_1 = ivar1[overlap_idx_1]

    overlap_idx_2 = np.where(wave2 <= wave1[-1]) 
    overlap_flux_2 = flux2[overlap_idx_2]
    overlap_ivar_2 = ivar2[overlap_idx_2]
    
    overlap_av_flux = inv_var_weighted_average(overlap_flux_1,overlap_ivar_1,overlap_flux_2,overlap_ivar_2)

    return overlap_av_flux, overlap_wave

# Stacking Functions 

In [4]:
#stacking functions

# Ignore warnings due to spec_var=0
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")


def get_spectra(table, column_name, search_values): 
    print('Number of spectra =', len(table))
    # Get the Target IDs as a numpy array
    column_data = table[column_name]

    SURVEY, PROGRAM, HEALPIX, n_values = [], [], [], []
    Z, RA, DEC, Source_ID = [], [], [], []


    for search_value in search_values:
        # Find the indices where the condition is met (value == search_value)
        # This is only relevant if input is doen by search value and not selected table
        indices = np.where(column_data == search_value)[0]

        if len(indices) > 0:
            # Find the corresponding Survey, program, healpix, Z , RA and Dec values
            SURVEY.append(table['SURVEY'][indices[0]])
            PROGRAM.append(table['PROGRAM'][indices[0]])
            HEALPIX.append(table['HEALPIX'][indices[0]])
            Z.append(table['Z'][indices[0]])
            RA.append(table['TARGET_RA'][indices[0]]) 
            DEC.append(table['TARGET_DEC'][indices[0]])
            Source_ID.append(table['TARGETID'][indices[0]])
            HEALPIX_str = str(table['HEALPIX'][indices[0]])
            n_str = HEALPIX_str[:3]
            n_values.append(int(n_str))
        else:
            # Handle the case where the search value was not found
            SURVEY.append(None)
            PROGRAM.append(None)
            HEALPIX.append(None)
            n_values.append(None)
            Z.append(None)   
            RA.append(None)
            DEC.append(None)
            Source_ID.append(None)
            
    
    z = np.array(Z)
    RA = np.array(RA)
    DEC = np.array(DEC)
    id_spec = np.array(Source_ID)
    wave_spec, flux_spec, sigma_spec = [], [], []
    
    for index, (survey, program, healpix, search_value) in enumerate(zip(SURVEY, PROGRAM, HEALPIX, search_values)):
        
        folder_name = 'DESI_Downloads'
        file_name = f"coadd-{survey}-{program}-{healpix}.fits"
        coadd_obj = desispec.io.read_spectra(f'{folder_name}/{file_name}')
        coadd_tgts = coadd_obj.target_ids().data
        # Selecting the particular spectra of the targetid
        row = (coadd_tgts == search_value)
        coadd_spec = coadd_obj[row] 

        # Inverse Vartiance weighted average for overlap
        no_overlap = no_overlap_f(coadd_spec.wave['b'],coadd_spec.flux['b'][0],coadd_spec.wave['r'],coadd_spec.flux['r'][0],coadd_spec.wave['z'],coadd_spec.flux['z'][0])
        overlap_br= overlap_av(coadd_spec.wave['b'],coadd_spec.wave['r'],coadd_spec.flux['b'][0],coadd_spec.flux['r'][0],coadd_spec.ivar['b'][0],coadd_spec.ivar['r'][0])
        overlap_rz= overlap_av(coadd_spec.wave['r'],coadd_spec.wave['z'],coadd_spec.flux['r'][0],coadd_spec.flux['z'][0],coadd_spec.ivar['r'][0],coadd_spec.ivar['z'][0])
        
        
        # Concaternating Spectra
        flux_spectra = np.concatenate((no_overlap[1],overlap_br[0],no_overlap[3],overlap_rz[0],no_overlap[5]))
        wave_spectra = np.concatenate((no_overlap[0],overlap_br[1],no_overlap[2],overlap_rz[1],no_overlap[4]))

        # Using Desispec to get wavelengths, flux and sigma
        wave = wave_spectra
        wave = wave.flatten()
        #flux = coadd_spec.flux['r'][0]
        #flux = convolve(flux_spectra, Gaussian1DKernel(5))
        #pick if i want to convolve or not
        flux = flux_spectra
        flux = flux.flatten()
        sigma = (np.sqrt(1/coadd_spec.ivar['r']))
        sigma = sigma.flatten()
        if index % 2 == 0 and len(wave) > 0:
                wave = wave[:-1]
                flux = flux[:-1]
                sigma = sigma[:-1]
        sigma_spec.append(sigma)
        wave_spec.append(wave)
        flux_spec.append(flux)
        
    
    wave_spec = np.array(wave_spec,dtype=object)
    flux_spec = np.array(flux_spec,dtype=object)
    sigma_spec = np.array(sigma_spec,dtype=object)
    # alternatively tried wave_spec = wave_spec.tolist()

    print('Number of spectra in wave spec=', len(wave_spec))
    
    return wave_spec, flux_spec, sigma_spec, z, RA, DEC, id_spec


def get_stack(spec, zbins=None):# can add redshift ranges i.e zbins = [[0.0,0.1],[0.1,0.2]]
    wave_spec, flux_spec, sigma_spec, z, RA, DEC, id_spec= spec
    stack = sc.stack_spectra(wave_spec,flux_spec,sigma_spec,
                               z, RA, DEC, zbins = zbins)
    return stack

# Stacking - Save to CSVs

In [ ]:
# Edit here to pick class
# LERG, HERG, RQAGN, SFG
Overall_class = 'LERG'







u_idx = len(crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'])
my_table = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'][0:u_idx]
search_column_name = 'TARGETID'
search_values = np.array(my_table[f'{search_column_name}'])


tble_1 = crossmatch[(crossmatch['Z']>0.03) & (crossmatch['Z'] < 0.4)]
my_table_1 = tble_1[tble_1['Overall_class'] == f'{Overall_class}']
tble_2 = crossmatch[(crossmatch['Z']>0.4) & (crossmatch['Z'] < 0.9)]
my_table_2 = tble_2[tble_2['Overall_class'] == f'{Overall_class}']
tble_3 = crossmatch[(crossmatch['Z']>0.9) & (crossmatch['Z'] < 1.6)] #1.6 as this is where redrock starts to fail
my_table_3 = tble_3[tble_3['Overall_class'] == f'{Overall_class}']

tables = [my_table, my_table_1, my_table_2, my_table_3]
stacks = []

for table in tables:
    search_values = np.array(table[f'{search_column_name}'])
    spec = get_spectra(table, search_column_name, search_values)
    stack = get_stack(spec)
    stacks.append(stack)

# Save stacks to CSV
for i, stack in enumerate(stacks):
    stack_df = pd.DataFrame({'wave_spec': stack['zbin=0']['wln'], 'flux_spec': stack['zbin=0']['flux']})
    stack_df.to_csv(f'{Overall_class}_stack_{i}.csv', index=False)
    print(f'Saved {Overall_class}_stack_{i}.csv')


To use the Mass or sSFR in SFG also use below too

In [5]:
# Calculate sSFR
crossmatch['sSFR_cons'] = crossmatch['SFR_cons'] - crossmatch['Mass_cons']

# Sort data by sSFR or mass
var_bin = 'sSFR_cons'
#var_bin = 'Mass_cons'
crossmatch = crossmatch[np.array(np.argsort(crossmatch[f'{var_bin}']))]




Overall_class = 'SFG'
crossmatch = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}']
u_idx = len(crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'])
all_data = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'][0:u_idx]
search_column_name = 'TARGETID'

tble_1 = crossmatch[(crossmatch['Z']>0.03) & (crossmatch['Z'] < 0.4)]
zbin_1 = tble_1[tble_1['Overall_class'] == f'{Overall_class}']
tble_2 = crossmatch[(crossmatch['Z']>0.4) & (crossmatch['Z'] < 0.9)]
zbin_2 = tble_2[tble_2['Overall_class'] == f'{Overall_class}']
tble_3 = crossmatch[(crossmatch['Z']>0.9) & (crossmatch['Z'] < 1.6)] #1.6 as this is where redrock starts to fail
zbin_3 = tble_3[tble_3['Overall_class'] == f'{Overall_class}']

tables = [all_data, zbin_1, zbin_2, zbin_3]

all_data_t = []
zbin_1_t = []
zbin_2_t = []
zbin_3_t = []
all_mass_max = []
all_mass_min = []
zbin_1_mass_max = []
zbin_1_mass_min = []
zbin_2_mass_max = []
zbin_2_mass_min = []
zbin_3_mass_max = []
zbin_3_mass_min = []

table_sizes = [6, 6, 4, 2]  # Number of bins for each table

for i, table in enumerate(tables):
    total_objects = len(table)
    table_size = total_objects // table_sizes[i]
    
    for j in range(table_sizes[i]):
        start_index = j * table_size
        end_index = (j + 1) * table_size if j < table_sizes[i] - 1 else total_objects
        
        # Create a new table for the current segment
        current_table = table[start_index:end_index]
        
        # Append the new table to the corresponding list based on the zbin
        if table is all_data:
            all_data_t.append(current_table)
            all_mass_max.append(np.max(current_table['Mass_cons']))
            all_mass_min.append(np.min(current_table['Mass_cons']))
        elif table is zbin_1:
            zbin_1_t.append(current_table)
            zbin_1_mass_max.append(np.max(current_table['Mass_cons']))
            zbin_1_mass_min.append(np.min(current_table['Mass_cons']))
        elif table is zbin_2:
            zbin_2_t.append(current_table)
            zbin_2_mass_max.append(np.max(current_table['Mass_cons']))
            zbin_2_mass_min.append(np.min(current_table['Mass_cons']))
        elif table is zbin_3:
            zbin_3_t.append(current_table)
            zbin_3_mass_max.append(np.max(current_table['Mass_cons']))
            zbin_3_mass_min.append(np.min(current_table['Mass_cons']))


"""

So zbin_1_t is a list of 6 tables


"""


'\n\nSo zbin_1_t is a list of 6 tables\n\n\n'

In [ ]:
stacks_zbin_1 = []

for table in zbin_1_t:
    search_values = np.array(table[f'{search_column_name}'])
    spec = get_spectra(table, search_column_name, search_values)
    stack = get_stack(spec)
    stacks_zbin_1.append(stack)

# Save stacks to CSV
for i, stack in enumerate(stacks_zbin_1):
    stack_df = pd.DataFrame({'wave_spec': stack['zbin=0']['wln'], 'flux_spec': stack['zbin=0']['flux']})
    stack_df.to_csv(f'zbin_1_stack_{i}.csv', index=False)
    print(f'Saved zbin_1_stack_{i}.csv')

In [ ]:
stacks_zbin_2 = []

for table in zbin_2_t:
    search_values = np.array(table[f'{search_column_name}'])
    spec = get_spectra(table, search_column_name, search_values)
    stack = get_stack(spec)
    stacks_zbin_2.append(stack)

# Save stacks to CSV
for i, stack in enumerate(stacks_zbin_2):
    stack_df = pd.DataFrame({'wave_spec': stack['zbin=0']['wln'], 'flux_spec': stack['zbin=0']['flux']})
    stack_df.to_csv(f'zbin_2_stack_{i}.csv', index=False)
    print(f'Saved zbin_2_stack_{i}.csv')

In [ ]:
stacks_zbin_3 = []

for table in zbin_3_t:
    search_values = np.array(table[f'{search_column_name}'])
    spec = get_spectra(table, search_column_name, search_values)
    stack = get_stack(spec)
    stacks_zbin_3.append(stack)

# Save stacks to CSV
for i, stack in enumerate(stacks_zbin_3):
    stack_df = pd.DataFrame({'wave_spec': stack['zbin=0']['wln'], 'flux_spec': stack['zbin=0']['flux']})
    stack_df.to_csv(f'zbin_3_stack_{i}.csv', index=False)
    print(f'Saved zbin_3_stack_{i}.csv')

# Read in CSV's

In [ ]:

"""

LERG / HERG / RQAGN



"""

Overall_class = 'LERG' # Change this to HERG or RQAGN



u_idx = len(crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'])
my_table = crossmatch[crossmatch['Overall_class'] == f'{Overall_class}'][0:u_idx]
search_column_name = 'TARGETID'

tble_1 = crossmatch[(crossmatch['Z']>0.03) & (crossmatch['Z'] < 0.4)]
my_table_1 = tble_1[tble_1['Overall_class'] == f'{Overall_class}']
tble_2 = crossmatch[(crossmatch['Z']>0.4) & (crossmatch['Z'] < 0.9)]
my_table_2 = tble_2[tble_2['Overall_class'] == f'{Overall_class}']
tble_3 = crossmatch[(crossmatch['Z']>0.9) & (crossmatch['Z'] < 1.6)] #1.6 as this is where redrock starts to fail
my_table_3 = tble_3[tble_3['Overall_class'] == f'{Overall_class}']

tables = [my_table, my_table_1, my_table_2, my_table_3]


print(len(tble_1), len(my_table_1))

stacks = []
wln = []
flux = []


for i in range(4):
    stack_df = pd.read_csv(f'{Overall_class}_stack_{i}.csv')
    stack = (stack_df['wave_spec'].values, stack_df['flux_spec'].values)
    wln.append(stack[0])
    flux.append(stack[1])
    stacks.append(stack)


wln_1, flux_1 = wln[1], flux[1]
wln_2, flux_2 = wln[2], flux[2]
wln_3, flux_3 = wln[3], flux[3]
wln, flux = wln[0], flux[0]

print(wln, flux)
print(wln_1, flux_1)
print(wln_2, flux_2)
print(wln_3, flux_3)

def renorm(flux, wln, wln_min, wln_max):
    """
    Normalizes the flux to the median wavelength in the specified range
    """
    indices = np.where((wln >= wln_min) & (wln <= wln_max))

    flux_norm = flux / np.median(flux[indices])
    
    return flux_norm

wln_min = 4500
wln_max = 4700

flux_1_norm = renorm(flux_1, wln_1, wln_min, wln_max)
flux_2_norm = renorm(flux_2, wln_2, wln_min, wln_max)
flux_3_norm = renorm(flux_3, wln_3, wln_min, wln_max)



"""

Add in some bounds to remove the upper and lower 10% of each spectra as noise reduction

"""

def bounds(stack_array):
        bound = 0.1
        l_bound = len(stack_array)*bound
        u_bound = len(stack_array)-(len(stack_array)*bound)

        return int(l_bound), int(u_bound)


st1 = bounds(wln_1)
st2 = bounds(wln_2)
st3 = bounds(wln_3)



In [ ]:


"""  

Read's CSV files back in - SFG

*** Need to change the name of what you are reading in depending on which stacks you want **

"""


def renorm(flux, wln, wln_min, wln_max):
    """
    Normalizes the flux to the median wavelength in the specified range
    """
    indices = np.where((wln >= wln_min) & (wln <= wln_max))

    flux_norm = flux / np.median(flux[indices])
    
    return flux_norm

wln_min = 4800
wln_max = 5000

all_data_stacks = []
all_data_wln = []
all_data_flux = []
all_data_flux_renorm = []

zbin_1_stacks = []
zbin_1_wln = []
zbin_1_flux = []    
zbin_1_flux_renorm = []

zbin_2_stacks = []
zbin_2_wln = []
zbin_2_flux = []
zbin_2_flux_renorm = []

zbin_3_stacks = []
zbin_3_wln = []
zbin_3_flux = []
zbin_3_flux_renorm = []

for i in range(6):
    #all_data_stack_df = pd.read_csv(f'all_data_stack_{i}.csv')
    #all_data_stack = (all_data_stack_df['wave_spec'].values, all_data_stack_df['flux_spec'].values)
    #all_data_stacks.append(all_data_stack)
    #all_data_wln.append(all_data_stack[0])
    #all_data_flux.append(all_data_stack[1])
    
    zbin_1_stack_df = pd.read_csv(f'zbin_1_stack_s{i}.csv') # Change here
    zbin_1_stack = (zbin_1_stack_df['wave_spec'].values, zbin_1_stack_df['flux_spec'].values)
    zbin_1_stacks.append(zbin_1_stack)
    zbin_1_wln.append(zbin_1_stack[0])
    zbin_1_flux.append(zbin_1_stack[1])
    zbin_1_flux_renorm.append(renorm(zbin_1_stack[1], zbin_1_stack[0], wln_min, wln_max))
    
for i in range(4):
    zbin_2_stack_df = pd.read_csv(f'zbin_2_stack_s{i}.csv') # Change here
    zbin_2_stack = (zbin_2_stack_df['wave_spec'].values, zbin_2_stack_df['flux_spec'].values)
    zbin_2_stacks.append(zbin_2_stack)
    zbin_2_wln.append(zbin_2_stack[0])
    zbin_2_flux.append(zbin_2_stack[1])
    zbin_2_flux_renorm.append(renorm(zbin_2_stack[1], zbin_2_stack[0], wln_min, wln_max))
    
for i in range(2):
    zbin_3_stack_df = pd.read_csv(f'zbin_3_stack_s{i}.csv') # Change here
    zbin_3_stack = (zbin_3_stack_df['wave_spec'].values, zbin_3_stack_df['flux_spec'].values)
    zbin_3_stacks.append(zbin_3_stack)
    zbin_3_wln.append(zbin_3_stack[0])
    zbin_3_flux.append(zbin_3_stack[1])
    zbin_3_flux_renorm.append(renorm(zbin_3_stack[1], zbin_3_stack[0], wln_min, wln_max))




# Equivalent Width Calculations (only applied to SFG)

In [ ]:
"""""
Continuum fit

Points to consider:
- Could change the region of wavelength to fit the continuum
- Could just fit polynomial to the continuum
- When exculing region need to cosider the N2 peaks


"""""


def fit_continuum(wln, flux, contin_min, contin_max, exclude_region, exclude_width):
    """
    Fit a continuum to the spectrum

    Parameters
    ----------
    wln : numpy.ndarray
        Wavelength array

    flux : numpy.ndarray
        Flux array

    contin_min : float
        Minimum wavelength to fit the continuum

    contin_max : float
        Maximum wavelength to fit the continuum

    exclude_region : float, optional
        Central wavelength of emission line to exclude from the continuum fit

    exclude_width : float, optional
        Width of the emission line to exclude from the continuum fit
    
    Returns
    -------
    y_continuum_fitted : numpy.ndarray
        Fitted continuum values
    
    wln_region : numpy.ndarray
        Wavelength array of the region to fit the continuum
    
    flux_region : numpy.ndarray
        Flux array of the region to fit the continuum

    spectrum : specutils.Spectrum1D
        Spectrum1D object of the region to fit the continuum
    

    """

    flux_region = flux[(wln >= contin_min) & (wln <= contin_max)]
    wln_region = wln[(wln >= contin_min) & (wln <= contin_max)]


    # Create a Spectrum1D object from the spectra
    spectrum = Spectrum1D(flux=flux_region*u.Unit('erg/s/cm^2/AA'), spectral_axis=wln_region*u.AA)

    # Define the region to exclude (H alpha peak)
    exclude_region = SpectralRegion(exclude_region*u.AA - exclude_width*u.AA, exclude_region*u.AA + exclude_width*u.AA)

    # Fit a generic continuum model to the spectrum, excluding the H alpha peak
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        g1_fit = fit_generic_continuum(spectrum, exclude_regions=[exclude_region])

    # Generate the fitted continuum values
    y_continuum_fitted = g1_fit(wln_region*u.AA)

    return y_continuum_fitted, wln_region, flux_region, spectrum

"""   
Get continuum Normalised and continuum subtracted spectra

Do I want to use the full spectrum or just the section I fitted the continuum to?

"""

def spec_norm(spectrum, cont):
    """
    Normalise the spectrum by dividing by the continuum

    Parameters
    ----------
    spectrum : numpy.ndarray
        Spectrum Values
        (output [2] from continuum fit function)

    cont : numpy.ndarray
        Fitted continuum values
        (output [0] from fit continuum function)
    
    Returns
    -------
    flux_norm.spectral_axis: numpy.ndarray
        Normalised wavelength values
    
    flux_norm.flux: numpy.ndarray
        Normalised flux values

    """

    flux_norm = spectrum / cont

    return flux_norm.spectral_axis, flux_norm.flux

def spec_sub(spectrum, cont):
    """
    Subtract the continuum from the spectrum

    Parameters
    ----------
    spectrum : numpy.ndarray
        Spectrum Values
        (output [2] from continuum fit function)

    cont : numpy.ndarray
        Fitted continuum values
        (output [0] from fit continuum function)
    
    Returns
    -------
    flux_sub.spectral_axis: numpy.ndarray
        Subtracted wavelength values
    
    flux_sub.flux: numpy.ndarray
        Subtracted flux values

    """

    flux_sub = spectrum - cont

    return flux_sub.spectral_axis, flux_sub.flux



def EW_integral(wln, flux, cont,wln_emissionline, line_width):
    """
    Calculate the equivalent width of a spectrum

    Parameters
    ----------
    wln : numpy.ndarray
        Wavelength array

    flux : numpy.ndarray
        Flux array

    cont : numpy.ndarray
        Fitted continuum values

    wln_emissionline : float
        Wavelength of the emission line
    
    line_width : float
        Width of the emission line
    
    Returns
    -------
    EW : float
        Equivalent width of the spectrum

    """

    # Define the region to integrate
    region = (wln >= wln_emissionline-line_width/2) & (wln <= wln_emissionline+line_width/2)

    # Calculate the equivalent width
    EW = np.trapz(1 - (flux[region] / cont[region]), wln[region])

    return EW

"""    
Fit Gaussian to H alpha peak
Find the Equivalent Width using FWHM*Amplitude

"""

from astropy.modeling import models
from astropy import units as u
from specutils.spectra import Spectrum1D
from specutils.fitting import fit_lines


def fit_gaussian(wln_sub, flux_sub, wln_emissionline, line_width):

    """
    Fit a Gaussian to the H alpha peak

    Parameters
    ----------
    wln_sub : numpy.ndarray
        Wavelength array of the continuum subtracted spectrum
    
    flux_sub : numpy.ndarray
        Flux array of the continuum subtracted spectrum
    
    wln_emissionline : float
        Wavelength of the emission line
    
    line_width : float
        Width of the emission line

    Returns
    -------
    y_fit : numpy.ndarray
        Fitted Gaussian values
    
    fwhm : float
        Full width at half maximum of the fitted Gaussian
    
    amplitude : float
        Amplitude of the fitted Gaussian
    
    equivalent_width : float
        Equivalent width of the fitted Gaussian

    wln_sub[region] : numpy.ndarray
        Wavelength array of the region to fit the Gaussian

    flux_sub[region] : numpy.ndarray
        Flux array of the region to fit the Gaussian
        
    """

    # Define the region to fit the Gaussian
    region = (wln_sub >= wln_emissionline-line_width/2) & (wln_sub <= wln_emissionline+line_width/2)

    # Create a Spectrum1D object covering emissionn line width
    spectrum_line = Spectrum1D(flux=flux_sub[region]*u.Unit('erg/s/cm^2/AA'), 
                    spectral_axis=wln_sub[region]*u.AA)
    
    # Fit a Gaussian model to the region of interest
    g_init = models.Gaussian1D(amplitude=np.max(spectrum_line.flux), mean=np.mean(spectrum_line.spectral_axis), stddev=1*u.AA)
    g_fit = fit_lines(spectrum_line, g_init)

    # Generate the fitted Gaussian values
    y_fit = g_fit(wln_sub[region]*u.AA)

    fwhm = g_fit.fwhm

    amplitude = g_fit.amplitude

    # Calculate the equivalent width 
    equivalent_width = fwhm*amplitude

    return y_fit, fwhm, amplitude, equivalent_width,  wln_sub[region], flux_sub[region]

    


In [ ]:
# H alpha for SFG

# naming used here is silly - make it more logical!



wln_emissionline = 6562.8
line_width = 20

ew_int_2 = []
ew_spec_2 = []
ew_gauss_2 = []

for i in range(4):
    y_continuum_fit, wln_region, flux_region, spectrum = fit_continuum(zbin_2_wln[i], zbin_2_flux[i], 6400, 6700, exclude_region = 6563, exclude_width = 100) # needs to be a wider gap excluded to exclude the N2 peaks
    flux_norm = spec_norm(spectrum, y_continuum_fit)
    flux_sub = spec_sub(spectrum, y_continuum_fit)
    eq_width_integral = EW_integral(wln_region, flux_region, y_continuum_fit.value, wln_emissionline, line_width)
    spectrum_norm = Spectrum1D(flux=flux_norm[1], spectral_axis=flux_norm[0])
    eq_width_spec = equivalent_width(spectrum_norm, regions=SpectralRegion((wln_emissionline-line_width/2)*u.AA, (wln_emissionline+line_width/2)*u.AA))
    y_fit, fwhm, amplitude, eq_width_gauss, wln_sub, flux_sub = fit_gaussian(flux_sub[0].value, flux_sub[1].value, wln_emissionline, line_width)
    ew_int_2.append(abs(eq_width_integral))
    ew_spec_2.append(abs(eq_width_spec.value))
    ew_gauss_2.append(abs(eq_width_gauss.value))

print(ew_int_2)
print(ew_spec_2)
print(ew_gauss_2)


ew_int_1 = []
ew_spec_1 = []
ew_gauss_1 = []

for i in range(6):
    y_continuum_fit, wln_region, flux_region, spectrum = fit_continuum(zbin_1_wln[i], zbin_1_flux[i], 6400, 6700, exclude_region = 6563, exclude_width = 100) # needs to be a wider gap excluded to exclude the N2 peaks
    flux_norm = spec_norm(spectrum, y_continuum_fit)
    flux_sub = spec_sub(spectrum, y_continuum_fit)
    eq_width_integral = EW_integral(wln_region, flux_region, y_continuum_fit.value, wln_emissionline, line_width)
    spectrum_norm = Spectrum1D(flux=flux_norm[1], spectral_axis=flux_norm[0])
    eq_width_spec = equivalent_width(spectrum_norm, regions=SpectralRegion((wln_emissionline-line_width/2)*u.AA, (wln_emissionline+line_width/2)*u.AA))
    y_fit, fwhm, amplitude, eq_width_gauss, wln_sub, flux_sub = fit_gaussian(flux_sub[0].value, flux_sub[1].value, wln_emissionline, line_width)
    ew_int_1.append(abs(eq_width_integral))
    ew_spec_1.append(abs(eq_width_spec.value))
    ew_gauss_1.append(abs(eq_width_gauss.value))

print(ew_int_1)
print(ew_spec_1)
print(ew_gauss_1)


In [ ]:
 ## O[II] for SFG


wln_emissionline = 3727
line_width = 20
from specutils.analysis import equivalent_width

ew_int_3 = []
ew_spec_3 = []
ew_gauss_3 = []

for i in range(4):
    y_continuum_fit, wln_region, flux_region, spectrum = fit_continuum(zbin_2_wln[i], zbin_2_flux[i], 3300, 4200, exclude_region = 6563, exclude_width = 100) # needs to be a wider gap excluded to exclude the N2 peaks
    flux_norm = spec_norm(spectrum, y_continuum_fit)
    flux_sub = spec_sub(spectrum, y_continuum_fit)
    eq_width_integral = EW_integral(wln_region, flux_region, y_continuum_fit.value, wln_emissionline, line_width)
    spectrum_norm = Spectrum1D(flux=flux_norm[1], spectral_axis=flux_norm[0])
    eq_width_spec = equivalent_width(spectrum_norm, regions=SpectralRegion((wln_emissionline-line_width/2)*u.AA, (wln_emissionline+line_width/2)*u.AA))
    y_fit, fwhm, amplitude, eq_width_gauss, wln_sub, flux_sub = fit_gaussian(flux_sub[0].value, flux_sub[1].value, wln_emissionline, line_width)
    ew_int_3.append(abs(eq_width_integral))
    ew_spec_3.append(abs(eq_width_spec.value))
    ew_gauss_3.append(abs(eq_width_gauss.value))

print(ew_int_3)
print(ew_spec_3)
print(ew_gauss_3)


ew_int_4 = []
ew_spec_4 = []
ew_gauss_4 = []

for i in range(6):
    y_continuum_fit, wln_region, flux_region, spectrum = fit_continuum(zbin_1_wln[i], zbin_1_flux[i], 3300, 4200, exclude_region = 6563, exclude_width = 100) # needs to be a wider gap excluded to exclude the N2 peaks
    flux_norm = spec_norm(spectrum, y_continuum_fit)
    flux_sub = spec_sub(spectrum, y_continuum_fit)
    eq_width_integral = EW_integral(wln_region, flux_region, y_continuum_fit.value, wln_emissionline, line_width)
    spectrum_norm = Spectrum1D(flux=flux_norm[1], spectral_axis=flux_norm[0])
    eq_width_spec = equivalent_width(spectrum_norm, regions=SpectralRegion((wln_emissionline-line_width/2)*u.AA, (wln_emissionline+line_width/2)*u.AA))
    y_fit, fwhm, amplitude, eq_width_gauss, wln_sub, flux_sub = fit_gaussian(flux_sub[0].value, flux_sub[1].value, wln_emissionline, line_width)
    ew_int_4.append(abs(eq_width_integral))
    ew_spec_4.append(abs(eq_width_spec.value))
    ew_gauss_4.append(abs(eq_width_gauss.value))

print(ew_int_4)
print(ew_spec_4)
print(ew_gauss_4)

ew_int_5 = []   
ew_spec_5 = []
ew_gauss_5 = []

for i in range(2):
    y_continuum_fit, wln_region, flux_region, spectrum = fit_continuum(zbin_3_wln[i], zbin_3_flux[i], 3300, 4200, exclude_region = 6563, exclude_width = 100) # needs to be a wider gap excluded to exclude the N2 peaks
    flux_norm = spec_norm(spectrum, y_continuum_fit)
    flux_sub = spec_sub(spectrum, y_continuum_fit)
    eq_width_integral = EW_integral(wln_region, flux_region, y_continuum_fit.value, wln_emissionline, line_width)
    spectrum_norm = Spectrum1D(flux=flux_norm[1], spectral_axis=flux_norm[0])
    eq_width_spec = equivalent_width(spectrum_norm, regions=SpectralRegion((wln_emissionline-line_width/2)*u.AA, (wln_emissionline+line_width/2)*u.AA))
    y_fit, fwhm, amplitude, eq_width_gauss, wln_sub, flux_sub = fit_gaussian(flux_sub[0].value, flux_sub[1].value, wln_emissionline, line_width)
    ew_int_5.append(abs(eq_width_integral))
    ew_spec_5.append(abs(eq_width_spec.value))
    ew_gauss_5.append(abs(eq_width_gauss.value))

print(ew_int_5)
print(ew_spec_5)
print(ew_gauss_5)


# Plotting Stacked Spectra

LERG / HERG / RQAGN

In [ ]:



# Create a PDF file to save the plots
pdf_filename = f'DESI-{Overall_class}-stack_Final.pdf'
with PdfPages(pdf_filename) as pdf:
        fig, ax = plt.subplots(figsize=(16, 5))
        ax.plot(wln,flux, color='blue', alpha=0.5,label = f'DESI Raw Stack, LOFAR Class {Overall_class}')
        ax.plot(wln,convolve(flux, Gaussian1DKernel(5)), color='black', alpha=1, label = f'DESI Smoothed Gaussian Stack, LOFAR Class {Overall_class}')
        ax.set_ylabel('$F_{\lambda}$ [$10^{-17} erg\ s^{-1}\ cm^{-2}\ \AA^{-1}$]')
        ax.set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]')
        ax.set_xlim(1200,8000)
        ax.set_ylim(-2,7)
        ax.set_xticks(range(1200, 8001, 500))
        ax.set_yticks(range(-2, 7, 1))
        ax.tick_params(axis='both', which='both', labelsize=12, direction = 'in')
        ax.minorticks_on()
        ax.legend(fontsize='large', loc = 'upper right')
        ax.xaxis.label.set_size(15)
        ax.yaxis.label.set_size(15)

     # Save the current plot to the PDF file
        pdf.savefig(fig)
        plt.show()
        plt.close()

# All the plots will be saved in the PDF file
print(f'All plots saved in {pdf_filename}')



In [ ]:
# Create a PDF file to save the plots
pdf_filename = f'DESI-{Overall_class}-stacked-zbin-plots-0.1cut-final.pdf'
with PdfPages(pdf_filename) as pdf:
        fig, ax = plt.subplots(figsize=(16, 5))
        ax.plot(wln_1[st1[0]:st1[1]],flux_1_norm[st1[0]:st1[1]], label = f'Raw Stack: 0.03<z<0.4', color='blue', alpha=0.5)
        ax.plot(wln_1[st1[0]:st1[1]],convolve(flux_1_norm, Gaussian1DKernel(5))[st1[0]:st1[1]], color='black', alpha=1)
        ax.plot(wln_2[st2[0]:st2[1]],flux_2_norm[st2[0]:st2[1]], label = f'Raw Stack: 0.4<z<0.9', color='green', alpha=0.5)
        ax.plot(wln_2[st2[0]:st2[1]],convolve(flux_2_norm, Gaussian1DKernel(5))[st2[0]:st2[1]], color='black', alpha=1)
        ax.plot(wln_3[st3[0]:st3[1]],flux_3_norm[st3[0]:st3[1]], label = f'Raw Stack: 0.9<z<1.6', color='red', alpha=0.5)
        ax.plot(wln_3[st3[0]:st3[1]],convolve(flux_3_norm, Gaussian1DKernel(5))[st3[0]:st3[1]], color='black', alpha=1, label = f'Smoothed Gaussian Stacks, \u03C3=5 ')
        ax.set_ylabel('$F_{\lambda, Norm}$ ')
        ax.set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]')
        ax.set_xlim(1800,8000)
        ax.set_ylim(-0.3,1.5)
        ax.set_xticks(range(1800, 8001, 500))
        ax.set_yticks(range(0, 2, 1))
        ax.tick_params(axis='both', which='both', labelsize=12, direction = 'in')
        ax.minorticks_on()
        ax.legend(fontsize='large', loc = 'upper left')
        ax.xaxis.label.set_size(15)
        ax.yaxis.label.set_size(15)
        fig.suptitle(f'DESI Spectra Stacks, LOFAR Class: {Overall_class}', fontsize=15)

     # Save the current plot to the PDF file
        pdf.savefig(fig)
        plt.show()
        plt.close()

# After the loop, all the plots will be saved in the PDF file
print(f'All plots saved in {pdf_filename}')

In [ ]:
"""    
Will need to change the x and y lims depending on the class being stcaked

"""



# Create a PDF file to save the plots
pdf_filename = f'DESI-{Overall_class}-stacked-zbin-plots-0.1cut_split_final.pdf'
with PdfPages(pdf_filename) as pdf:
    # Create a 1x3 grid of subplots with no spacing between plots
    fig, axs = plt.subplots(1, 3, figsize=(33,8), sharey=True, sharex=False, gridspec_kw={'wspace': 0})
    
    # Plot for column 1 (flux_1)
    axs[0].plot(wln_1[st1[0]:st1[1]], flux_1_norm[st1[0]:st1[1]], label=f'Raw Stack', color='blue', alpha=0.5)
    axs[0].plot(wln_1[st1[0]:st1[1]], convolve(flux_1_norm, Gaussian1DKernel(5))[st1[0]:st1[1]], color='black', alpha=1, label=f'Smoothed Gaussian Stack, \u03C3=5 ')
    axs[0].set_xlim(3400, 8400)
    axs[0].set_xticks(range(3400, 8399, 1000))
    axs[0].set_title('0.03<z<0.4', fontsize=20)
    
    # Plot for column 2 (flux_2)
    axs[1].plot(wln_2[st2[0]:st2[1]], flux_2_norm[st2[0]:st2[1]], label=f'Raw Stack', color='green', alpha=0.5)
    axs[1].plot(wln_2[st2[0]:st2[1]], convolve(flux_2_norm, Gaussian1DKernel(5))[st2[0]:st2[1]], color='black', alpha=1, label=f'Smoothed Gaussian Stack, \u03C3=5 ')
    axs[1].set_xlim(2500, 6400)
    axs[1].set_xticks(range(2500, 6401, 1000))
    axs[1].set_title('0.4<z<0.9', fontsize=20)

    # Plot for column 3 (flux_3)
    axs[2].plot(wln_3[st3[0]:st3[1]], flux_3_norm[st3[0]:st3[1]], label=f'Raw Stack', color='red', alpha=0.5)
    axs[2].plot(wln_3[st3[0]:st3[1]], convolve(flux_3_norm, Gaussian1DKernel(5))[st3[0]:st3[1]], color='black', alpha=1, label=f'Smoothed Gaussian Stack, \u03C3=5 ')
    axs[2].set_xlim(1800, 4300)
    axs[2].set_xticks(range(1800, 4301, 500))
    axs[2].set_title('0.9<z<1.6', fontsize=20)

    # Customize common settings for all subplots
    for i, ax in enumerate(axs):
        ax.set_ylabel('$F_{\lambda, Norm}$ ')
        ax.set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]')
        #ax.set_xlim(1100, 8500)
        ax.set_ylim(-0.3, 1.5)
        #ax.set_xticks(range(1100, 8501, 1000))  # Adjust the tick values
        ax.set_yticks(range(0, 2, 1))
        ax.tick_params(axis='both', which='both', labelsize=12, direction='in')
        ax.minorticks_on()
        if i != 0:
            ax.set_ylabel('')
        ax.legend(fontsize='large', loc='upper left')  # Adjust the legend location
        ax.xaxis.label.set_size(15)
        ax.yaxis.label.set_size(15)
    
    fig.suptitle(f'DESI Spectra Stacks, LOFAR Class: {Overall_class}', fontsize=25)

    
    # Save the current plot to the PDF file
    pdf.savefig(fig)
    plt.show()
    plt.close()

# After the loop, all the plots will be saved in the PDF file
print(f'All plots saved in {pdf_filename}')



SFG

In [ ]:


font = 20
print('Range for All data:', all_mass_min,all_mass_max)
print('Range for zbin_1:', zbin_1_mass_min,zbin_1_mass_max)
print('Range for zbin_2:', zbin_2_mass_min,zbin_2_mass_max)
print('Range for zbin_3:', zbin_3_mass_min,zbin_3_mass_max)



# Plotting for zbin 1
fig, axes = plt.subplots(6, 3, figsize=(30, 30), sharex=False, sharey=False)

"""
****THIS PLOT IS CURRENTLY RENORMALISED TO 4800-5000 RANGE***

"""


# Define the alpha values for each row
alpha_values = [0.2, 0.3, 0.4, 0.5, 0.7, 1.0][::-1]


for i in range(6):
        axes[5-i, 0].set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]', fontsize= font)
        axes[5-i, 0].set_ylabel('$F_{\lambda, Norm}$ ', fontsize= font)
        #axes[5-i, 0].legend(fontsize=12, loc='upper left')
        axes[5-i, 0].set_xlim(2700, 8000)
        axes[5-i, 0].set_ylim(-0.3, 4.5)
        axes[5-i, 0].tick_params(axis='both', which='major', labelsize=font) 
        text_combined = (f'Log (M/M$\odot$) = {zbin_1_mass_min[i]:.2f}:{zbin_1_mass_max[i]:.2f}\n' +
                    f'H$\\alpha$ Equivalent Width [$\AA$] = {abs(ew_int_1[i]):.2f}\n' +
                    f'[OII] Equivalent Width [$\AA$] = {abs(ew_int_4[i]):.2f}')
        axes[5-i, 0].text(0.02, 0.95, text_combined, transform=axes[5-i, 0].transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))
        
        
        if i != 0:
            axes[5-i, 0].plot(zbin_1_wln[i], zbin_1_flux_renorm[i], color='dodgerblue', alpha=alpha_values[i])
            axes[5-i, 0].plot(zbin_1_wln[i], convolve(zbin_1_flux_renorm[i], Gaussian1DKernel(5)),color='black', alpha=1.0)
            axes[5-i, 0].plot(zbin_1_wln[0], convolve(zbin_1_flux_renorm[0], Gaussian1DKernel(5)),color='black',linestyle = '--', alpha=0.6)
        else:
            axes[5-i, 0].plot(zbin_1_wln[i], zbin_1_flux_renorm[i], 
                                label = f'Raw Stacks  0.03<z<0.4', color='dodgerblue', alpha=alpha_values[i])
            axes[5-i, 0].plot(zbin_1_wln[i], convolve(zbin_1_flux_renorm[i], Gaussian1DKernel(5)),
                                color='black', alpha=1.0)
            axes[5-i, 0].plot(zbin_1_wln[0], convolve(zbin_1_flux_renorm[0], Gaussian1DKernel(5)),
                                color='black',linestyle = '--', alpha=0.6)
            

# Plotting for zbin 2
for i in range(4):
    axes[5-i, 1].set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]', fontsize= font)
    axes[5-i, 1].set_ylabel('$F_{\lambda, Norm}$ ', fontsize= font)
    #axes[5-i, 1].legend(fontsize=12, loc='upper left')
    axes[5-i, 1].set_xlim(2000, 7000)
    axes[5-i, 1].set_ylim(-0.3, 4.5)
    axes[5-i, 1].tick_params(axis='both', which='major', labelsize=font)  
    text_combined = (f'Log (M/M$\odot$) = {zbin_2_mass_min[i]:.2f}:{zbin_2_mass_max[i]:.2f}\n' +
                 f'H$\\alpha$ Equivalent Width [$\AA$] = {abs(ew_int_2[i]):.2f}\n'+
                 f'[OII] Equivalent Width [$\AA$] = {abs(ew_int_3[i]):.2f}')
    axes[5-i, 1].text(0.02, 0.95, text_combined, transform=axes[5-i, 1].transAxes, fontsize=10,
                  verticalalignment='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

    if i != 0:
        axes[5-i, 1].plot(zbin_2_wln[i], zbin_2_flux_renorm[i], color='forestgreen', alpha=alpha_values[i])
        axes[5-i, 1].plot(zbin_2_wln[i], convolve(zbin_2_flux_renorm[i], Gaussian1DKernel(5)), color='black', alpha=1.0)
        axes[5-i, 1].plot(zbin_2_wln[0], convolve(zbin_2_flux_renorm[0], Gaussian1DKernel(5)), color='black',linestyle = '--', alpha=0.6)
    else:
        axes[5-i, 1].plot(zbin_2_wln[i], zbin_2_flux_renorm[i], 
                        label = f'Raw Stacks 0.4<z<0.9 ', color='forestgreen', alpha=alpha_values[i])
        axes[5-i, 1].plot(zbin_2_wln[i], convolve(zbin_2_flux_renorm[i], Gaussian1DKernel(5)),
                        color='black', alpha=1.0)
        axes[5-i, 1].plot(zbin_2_wln[0], convolve(zbin_2_flux_renorm[0], Gaussian1DKernel(5)),
                        color='black',linestyle = '--', alpha=0.6)
    

# Plotting for zbin 3
for i in range(2):
    axes[5-i, 2].set_xlabel('Rest-frame wavelength, $\lambda$ [$\AA$]', fontsize= font)
    axes[5-i, 2].set_ylabel('$F_{\lambda, Norm}$ ', fontsize= font)
    #axes[5-i, 2].legend(fontsize=12, loc='upper left')
    axes[5-i, 2].set_xlim(1500, 5000)
    axes[5-i, 2].set_ylim(-0.3, 4.5)
    axes[5-i, 2].tick_params(axis='both', which='major', labelsize=font) 
    text_combined = (f'Log (M/M$\odot$) = {zbin_3_mass_min[i]:.2f}:{zbin_3_mass_max[i]:.2f}\n' +
                    f'[OII] Equivalent Width [$\AA$] = {abs(ew_int_5[i]):.2f}')
    axes[5-i, 2].text(0.02, 0.95, text_combined, transform=axes[5-i, 2].transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))
 

    if i!= 0:
        axes[5-i, 2].plot(zbin_3_wln[i], zbin_3_flux_renorm[i], color='indianred', alpha=alpha_values[i])
        axes[5-i, 2].plot(zbin_3_wln[i], convolve(zbin_3_flux_renorm[i], Gaussian1DKernel(5)), color='black', alpha=1.0)
        axes[5-i, 2].plot(zbin_3_wln[1], convolve(zbin_3_flux_renorm[1], Gaussian1DKernel(5)), color='black', linestyle = '--', alpha=0.6)
    else:
        axes[5-i, 2].plot(zbin_3_wln[i], zbin_3_flux_renorm[i], 
                        label = f'Raw Stacks 0.9<z<1.6 ', color='indianred', alpha=alpha_values[i])
        axes[5-i, 2].plot(zbin_3_wln[i], convolve(zbin_3_flux_renorm[i], Gaussian1DKernel(5)),
                        label = f'Smoothed Gaussian Stack ', color='black', alpha=1.0)
        axes[5-i, 2].plot(zbin_3_wln[1], convolve(zbin_3_flux_renorm[1], Gaussian1DKernel(5)),
                        label = f'Lowest Mass Bin Smoothed Gaussian Stacks', color='black', linestyle = '--', alpha=0.4)
        

# Remove x-axis label on all plots except row 5
for i in range(6):
    for j in range(3):
        if i != 5:
            axes[i, j].set_xlabel('')
            axes[i, j].tick_params(axis='x', which='both', labelsize=0)

# Remove y-axis label on all plots except column 0
for i in range(6):
    for j in range(3):
        if j != 0:
            axes[i, j].set_ylabel('')
            axes[i, j].tick_params(axis='y', which='both', labelsize=0)

# Delete rows 0, 1, 2, 3 in column 2
for i in range(4):
    fig.delaxes(axes[i, 2])

# Delete rows 0, 1 in column 1
for i in range(2):
    fig.delaxes(axes[i, 1])

plt.subplots_adjust(top = 0.95,hspace=.0, wspace=.0)

fig.suptitle('DESI Spectra Stacks, LOFAR Class: SFG', fontsize=30)
axes[0, 0].set_title('0.03<z<0.4', fontsize=20)
axes[2, 1].set_title('0.4<z<0.9', fontsize=20)
axes[4, 2].set_title('0.9<z<1.6', fontsize=20)
fig.legend(fontsize=20, loc='upper right')


plt.show()


